# Data Science Unit 4 Sprint Challenge 1 — Tree Ensembles

### Chicago Food Inspections

For this Sprint Challenge, you'll use a dataset with information from inspections of restaurants and other food establishments in Chicago from January 1, 2010 to the present. 

[See this PDF](https://data.cityofchicago.org/api/assets/BAD5301B-681A-4202-9D25-51B2CAE672FF) for descriptions of the data elements included in this dataset.

According to [Chicago Department of Public Health — Food Protection Services](https://www.chicago.gov/city/en/depts/cdph/provdrs/healthy_restaurants/svcs/food-protection-services.html), "Chicago is home to 16,000 food establishments like restaurants, grocery stores, bakeries, wholesalers, lunchrooms, mobile food vendors and more. Our business is food safety and sanitation with one goal, to prevent the spread of food-borne disease. We do this by inspecting food businesses, responding to complaints and food recalls." 

#### Your challenge: Predict whether inspections failed

The target is the `Fail` column.

- When the food establishment failed the inspection, the target is `1`.
- When the establishment passed, the target is `0`.

#### Run this cell to load the data:

In [30]:
import pandas as pd
import numpy as np
import category_encoders as ce
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from collections import defaultdict
plt.style.use('dark_background')

train_url = 'https://drive.google.com/uc?export=download&id=13_tP9JpLcZHSPVpWcua4t2rY44K_s4H5'
test_url  = 'https://drive.google.com/uc?export=download&id=1GkDHjsiGrzOXoF_xcYjdzBTSjOIi3g5a'

train = pd.read_csv(train_url).sample(2**13)
test  = pd.read_csv(test_url).sample(2**12)

X_train_0 = train.drop(['Fail'], axis=1)
X_test_0 = test.drop(['Fail'], axis=1)
y_train = train.Fail
y_test = test.Fail

#assert train.shape == (51916, 17)
#assert test.shape  == (17306, 17)

### Part 1: Preprocessing

You may choose which features you want to use, and whether/how you will preprocess them. You may use any tools and techniques for categorical encoding. (Pandas, category_encoders, sklearn.preprocessing, or any other library.)

_To earn a score of 3 for this part, engineer new features, and use any alternative categorical encoding instead of One-Hot or Ordinal/Label encoding._

### Part 2: Modeling

Fit a Random Forest or Gradient Boosting model with the train set. (You may use scikit-learn, xgboost, or any other library.) Use cross-validation to estimate an ROC AUC validation score.

Use your model to predict probabilities for the test set. Get an ROC AUC test score >= 0.60.

_To earn a score of 3 for this part, get an ROC AUC test score >= 0.70._


### Part 3: Visualization

Make one visualization for model interpretation. (You may use any libraries.) Choose one of these types:
- Feature Importances
- Permutation Importances
- Partial Dependence Plot

_To earn a score of 3 for this part, make at least two of these visualization types._

In [31]:
train.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Violations,Latitude,Longitude,Location,Fail
28530,2078473,DECOLORES RESTAURANT,DECOLORES RESTAURANT,2542897.0,Restaurant,Risk 1 (High),1626 S HALSTED ST,CHICAGO,IL,60608.0,2017-08-15T00:00:00,License,NaN,41.859183,-87.646830,"{'longitude': '-87.64682982484355', 'latitude'...",0
34898,1575529,HAPPY FISH & CHICKEN,HAPPY FISH & CHICKEN,2137607.0,Restaurant,Risk 1 (High),5-7 S CICERO AVE,CHICAGO,IL,60644.0,2015-09-14T00:00:00,Canvass,"11. ADEQUATE NUMBER, CONVENIENT, ACCESSIBLE, D...",41.880312,-87.745124,"{'longitude': '-87.74512375356417', 'latitude'...",1
3603,277479,FOUR MOON TAVERN,FOUR MOON TAVERN,51644.0,Restaurant,Risk 1 (High),1847 W ROSCOE ST,CHICAGO,IL,60657.0,2010-07-16T00:00:00,Canvass Re-Inspection,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",41.943141,-87.675707,"{'longitude': '-87.67570739466916', 'latitude'...",0
20187,1424401,PLUSH PUP,PLUSH PUP,1647850.0,Restaurant,Risk 1 (High),5344 N CUMBERLAND AVE,CHICAGO,IL,60656.0,2014-07-31T00:00:00,Canvass,"16. FOOD PROTECTED DURING STORAGE, PREPARATION...",41.976851,-87.836760,"{'longitude': '-87.83676047190316', 'latitude'...",1
37813,277935,Mama Mia Chicago Pizza,Mama Mia Chicago Pizza,2009418.0,Restaurant,Risk 1 (High),3131 N CENTRAL AVE,CHICAGO,IL,60634.0,2010-08-03T00:00:00,Short Form Complaint,NaN,41.937695,-87.766319,"{'longitude': '-87.76631878609123', 'latitude'...",1


In [32]:
train.dtypes

Inspection ID        int64
DBA Name            object
AKA Name            object
License #          float64
Facility Type       object
Risk                object
Address             object
City                object
State               object
Zip                float64
Inspection Date     object
Inspection Type     object
Violations          object
Latitude           float64
Longitude          float64
Location            object
Fail                 int64
dtype: object

In [33]:
train.describe()

,Inspection ID,License #,Zip,Latitude,Longitude,Fail
count,8.192000e+03,8.192000e+03,8189.000000,8158.000000,8158.000000,8192.000000
mean,1.329727e+06,1.543271e+06,60629.137135,41.880504,-87.677104,0.255859
std,6.186019e+05,9.085727e+05,21.097429,0.080876,0.060136,0.436370
min,4.822000e+04,0.000000e+00,60153.000000,41.649558,-87.914428,0.000000
25%,6.796932e+05,1.095793e+06,60614.000000,41.831172,-87.708498,0.000000
50%,1.424408e+06,1.963807e+06,60625.000000,41.890829,-87.667064,0.000000
75%,1.946848e+06,2.215542e+06,60643.000000,41.939783,-87.634281,1.000000
max,2.279423e+06,3.808691e+06,60827.000000,42.021064,-87.525094,1.000000


In [34]:
X_train_0.isna().sum()

Inspection ID         0
DBA Name              0
AKA Name             93
License #             0
Facility Type        27
Risk                  3
Address               0
City                  5
State                 0
Zip                   3
Inspection Date       0
Inspection Type       0
Violations         1561
Latitude             34
Longitude            34
Location             34
dtype: int64

In [35]:
todrop1 = ['Inspection ID', 'AKA Name', 'License #', 'Location']

def wrangle(dat): 
    import datetime as dt
    assigns = {
        **{'Violations': dat.Violations.fillna('NO_VIOLATIONS'), 
          'Facility Type': dat['Facility Type'].fillna('NOT_APPLICABLE'), 
          'Inspection Date': pd.to_datetime(dat['Inspection Date'], 
                                            infer_datetime_format=True
                                           ).apply(dt.datetime.toordinal)}, 
        **{l: dat[l].fillna(dat[l].mean()) for l in ['Latitude', 'Longitude']}
    }
    
    return dat.assign(**assigns).drop(todrop1, axis=1)

X_train_ = wrangle(X_train_0)
X_test_ = wrangle(X_test_0)

In [36]:
X_train_.isna().sum()

DBA Name           0
Facility Type      0
Risk               3
Address            0
City               5
State              0
Zip                3
Inspection Date    0
Inspection Type    0
Violations         0
Latitude           0
Longitude          0
dtype: int64

In [37]:
print(f'The target is unbalanced target by a factor of {y_train.value_counts()[0] / y_train.value_counts()[1]} favoring `0`. \n\n')

cats = X_train_.select_dtypes(include='object').columns
nums = X_train_.select_dtypes(exclude='object').columns

print(f"We have {len(cats)} categoricals and {len(nums)} numerics. \n")

cards = {n: X_train_[n].value_counts().shape[0] for n in cats}

def report_cards(c): 
    l = [f"\tFeature {k} has cardinality {v}. " for k,v in c.items()]
    return '\n'.join(l)

print(report_cards(cards))

The target is unbalanced target by a factor of 2.9083969465648853 favoring `0`. 


We have 8 categoricals and 4 numerics. 

	Feature DBA Name has cardinality 5943. 
	Feature Facility Type has cardinality 139. 
	Feature Risk has cardinality 3. 
	Feature Address has cardinality 5743. 
	Feature City has cardinality 10. 
	Feature State has cardinality 1. 
	Feature Inspection Type has cardinality 28. 
	Feature Violations has cardinality 6630. 


In [38]:
## We're going to encode "high" and "low" cardinality featuers differently, depending on some meaning of "high"/"low"

thresh=2**12
low = [k for k in cards.keys() if cards[k]<=thresh]
high = [k for k in cards.keys() if cards[k]>thresh]

assert len(low) + len(high) == len(cards.keys())

# send lows to binary encoder and send highs to target encoder. 
low, high

(['Facility Type', 'Risk', 'City', 'State', 'Inspection Type'],
 ['DBA Name', 'Address', 'Violations'])

In [39]:
def encode(train_dat, test_dat, low=low, high=high, targ=y_train, n=4): 
    '''
    base2 encode low cardinality, target encode high cardinailty
    
    Having given up on Target encoding because of `IndexingError: 
        Unalignable boolean Series provided as indexer`, 
    we shall basen encode even the high cardinality ones.   
    
    
    DO NOT RUN THIS. It crashes the computer. 
        #plt.plot(x=list(range(1,10)), y=[encode(X_train[cats], n=k) for k in range(1,10)])
    '''
    assert [x=='object' for x in train_dat.dtypes]
    assert [x=='object' for x in test_dat.dtypes]
    assert all([x==y for x,y in zip(X_train_[high].index, y_train.index)])
    
    bl = ce.basen.BaseNEncoder(base=n, return_df=True)
    #th = ce.TargetEncoder()
    
    bl.fit(train_dat[low+high])
    #bl_df = bl.fit_transform(train_dat[low+high])
    #th_df = th.fit_transform(dat[high], targ, return_df=True)
    
    td = bl.transform(train_dat[low+high])
    print(f"Now is {td.shape[1]} really too many columns?")
    return {'train': td, 'test': bl.transform(test_dat[low+high])}


In [40]:
# cats_dfs = encode(X_train_[cats], X_test_[cats])
# X_train = pd.concat([X_train_[nums], cats_dfs['train']], axis=1, sort=False)
# X_test = pd.concat([X_test_[nums], cats_dfs['test']], axis=1, sort=False)

# X_train.head()

In [41]:
# dtrain = xgb.DMatrix(X_train.values, y_train.values)
# dtest = xgb.DMatrix(X_test.values) 

# # specify parameters via map
# param = {'booster': 'dart',
#          'max_depth': 5, 'learning_rate': 0.1,
#          'objective': 'binary:logistic', 
#          'silent': True,
#          'sample_type': 'uniform',
#          'normalize_type': 'tree',
#          'rate_drop': 0.1,
#          'skip_drop': 0.5}

# num_round = 50
# bst = xgb.train(param, dtrain, num_round)
# # make prediction
# # ntree_limit must not be 0
# preds = bst.predict(dtest, ntree_limit=num_round)

# preds_df = pd.DataFrame({'id': X_test.index, 'failure': preds})
# #preds_df.head()

# roc_auc_score(y_test, preds_df['failure'])

In [50]:
def make_dmats(train=X_train_, test=X_test_, n=3): 
    cats_dfs = encode(train[cats], test[cats], n=n)
    X_train = pd.concat([train[nums], cats_dfs['train']], axis=1, sort=False)
    X_test = pd.concat([test[nums], cats_dfs['test']], axis=1, sort=False)
    
    dtrain = xgb.DMatrix(X_train.values, y_train.values)
    dtest = xgb.DMatrix(X_test.values) 

    return dtrain, dtest

def dart_score(dtrain, dtest, max_depth=5): 
    '''run a dart model and return roc_auc score '''
    
    # specify parameters via map
    param = {'booster': 'dart',
             'max_depth': max_depth, 
             'learning_rate': 0.1,
             'objective': 'binary:logistic', 
             'silent': True,
             'sample_type': 'uniform',
             'normalize_type': 'tree',
             'rate_drop': 0.1,
             'skip_drop': 0.5}

    num_round = 50
    bst = xgb.train(param, dtrain, num_round)
    # make prediction
    # ntree_limit must not be 0
    preds = bst.predict(dtest, ntree_limit=num_round)

    preds_df = pd.DataFrame({'id': X_test_.index, 'failure': preds})

    return roc_auc_score(y_test, preds_df['failure'])

def vanilla_boost_score(dtrain, dtest, max_depth=2, eta=0.8):
    # specify parameters via map
    param = {'max_depth': max_depth, 
             'eta': eta, 
             'silent':1, 
             'objective':'binary:logistic' }
    num_round = 2
    bst = xgb.train(param, dtrain, num_round)
    # make prediction
    preds = bst.predict(dtest)
    preds_df = pd.DataFrame({'id': X_test_.index, 'failure': preds})
    return roc_auc_score(y_test, preds_df['failure'])

dart_choices_n_coder_rocauc = {}
vani_choices_n_coder_rocauc = {}
for k in [2,3,4,5,6,7,8]:
    dmats = make_dmats(n=k)
    dart_choices_n_coder_rocauc[k] = dart_score(dmats[0], dmats[1])
    #vani_choices_n_coder_rocauc[k] = vanilla_boost_score(dmats[0], dmats[1])

dart_choices_n_coder_rocauc#, vani_choices_n_coder_rocauc

Now is 66 really too many columns?
Now is 47 really too many columns?
Now is 39 really too many columns?
Now is 36 really too many columns?
Now is 31 really too many columns?
Now is 31 really too many columns?
Now is 31 really too many columns?


{2: 0.677407998783455,
 3: 0.44091909975669097,
 4: 0.6376748783454989,
 5: 0.4360270681265207,
 6: 0.661760796836983,
 7: 0.6771511557177615,
 8: 0.6537320559610705}

In [43]:
# The dart at n=2 is the best, it appears. 


### Part 3: Visualization

Make one visualization for model interpretation. (You may use any libraries.) Choose one of these types:
- Feature Importances
- Permutation Importances
- Partial Dependence Plot

_To earn a score of 3 for this part, make at least two of these visualization types._

In [44]:
from eli5.xgboost import explain_prediction_xgboost
from eli5 import show_weights

from xgboost import XGBClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

# clf = XGBClassifier()
# vec = DictVectorizer()
# pipeline = make_pipeline(vec, clf)

# def evaluate(_clf):
#     scores = cross_val_score(_clf, all_xs, all_ys, scoring='accuracy', cv=10)
#     print('Accuracy: {:.3f} ± {:.3f}'.format(np.mean(scores), 2 * np.std(scores)))
#     _clf.fit(train_xs, train_ys)  # so that parts of the original pipeline are fitted

# evaluate(pipeline)


# def dart(dtrain, dtest, max_depth=5): 
#     '''run a dart model and return bst object '''
    
#     # specify parameters via map
#     param = {'booster': 'dart',
#              'max_depth': max_depth, 
#              'learning_rate': 0.1,
#              'objective': 'binary:logistic', 
#              'silent': True,
#              'sample_type': 'uniform',
#              'normalize_type': 'tree',
#              'rate_drop': 0.1,
#              'skip_drop': 0.5}

#     num_round = 50
#     bst = xgb.train(param, dtrain, num_round)
# #     # make prediction
# #     # ntree_limit must not be 0
# #     preds = bst.predict(dtest, ntree_limit=num_round)

# #     preds_df = pd.DataFrame({'id': X_test_.index, 'failure': preds})

#     return bst # roc_auc_score(y_test, preds_df['failure'])



In [45]:
# dmats = make_dmats(n=2)
# #explain_prediction_xgboost(dart(dmats[0], dmats[1]), doc=)

# pipeline=make_pipeline(vec, dart(dmats[0], dmats[1]))

# evaluate(pipeline)

In [46]:
#help(explain_prediction_xgboost)

In [48]:
try_ns = [2,7]

cats_dfs = encode(train[cats], test[cats], n=2)

mod = xgb.XGBClassifier()

mod.fit(cats_dfs['train'], y_train)

mod.feature_importances_

Now is 66 really too many columns?


(66,)